# **Install Required python library**

In [1]:
!pip install --force tqdm==4.41.1


     |████████████████████████████████| 61kB 9.0MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
!pip install flask

# **Import Library**

In [2]:
import flask
from IPython import get_ipython
import os

In [3]:
!rm -rf sample_data
!rm -rf stylegan
!git clone https://github.com/mahrach8/stylegan-encoder.git
!mv stylegan-encoder stylegan

Cloning into 'stylegan-encoder'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 502 (delta 7), reused 11 (delta 4), pack-reused 487
Receiving objects: 100% (502/502), 12.71 MiB | 34.90 MiB/s, done.
Resolving deltas: 100% (273/273), done.


In [4]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


Nous allons tout d'abord importer le repo git qui permet de trouver le vecteur latent qui correpond à un visage.

In [5]:
cd stylegan

/content/stylegan


In [6]:
!gdown https://drive.google.com/uc?id=1aT59NFy9-bNyXjDuZOTMl0qX0jmZc6Zb
!mkdir data
!mv finetuned_resnet.h5 data
!rm -rf images_generated latent_representations

Downloading...
From: https://drive.google.com/uc?id=1aT59NFy9-bNyXjDuZOTMl0qX0jmZc6Zb
To: /content/stylegan/finetuned_resnet.h5
330MB [00:01, 167MB/s]


In [7]:
mkdir raw_images

Il faudra mettre les images que l'on veut encoder dans le dossier images_source

# **Download ngrok and extract it**

In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2021-02-11 21:25:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.202.112.21, 54.85.41.146, 3.216.229.131, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.202.112.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  46.3MB/s    in 0.3s    

2021-02-11 21:25:18 (46.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


#**GAN Stuff**

In [21]:
import shutil
def reset_folder(path):
  if os.path.exists(path):
    if (len(os.listdir(path)) != 0):
      shutil.rmtree(path)
      os.mkdir(path)
  else:
    os.mkdir(path)

In [10]:
def generate_images(generator, latent_vector, z = True):
    batch_size = latent_vector.shape[0]
    
    if z: #Start from z: run the full generator network
        return generator.run(latent_vector.reshape((batch_size, 512)), None, randomize_noise=False, **synthesis_kwargs)
    else: #Start from w: skip the mapping network
        return generator.components.synthesis.run(latent_vector.reshape((batch_size, 18, 512)), randomize_noise=False, **synthesis_kwargs)

In [11]:
import align_images
import encode_images
import os
import dnnlib, pickle
import numpy as np
from PIL import Image
import dnnlib.tflib as tflib

synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=1)

def morphing():

  tflib.init_tf()

  reset_folder("aligned_images")
  reset_folder("generated_images")
  reset_folder("latent_representations")

  align_images.main("raw_images","aligned_images")
  
  print("images_aligned contient %d images" %len(os.listdir('aligned_images/')))
  
  encode_images.main("aligned_images", "generated_images", "latent_representations")

  model_dir = 'cache/'
  model_path = [model_dir+f for f in os.listdir(model_dir) if 'stylegan-ffhq' in f][0]
  print("Loading StyleGAN model from %s..." %model_path)

  with dnnlib.util.open_url(model_path) as f:
    generator_network, discriminator_network, averaged_generator_network = pickle.load(f)
    
  print("StyleGAN loaded & ready for sampling!")

  for f in sorted(os.listdir('latent_representations')):
    w = np.load('latent_representations/' + f).reshape((1,18,-1))
    img = generate_images(averaged_generator_network, w, z = False)[0]
  
  latents = sorted(os.listdir('latent_representations'))

  first = np.load('latent_representations/' + latents[0]).reshape((1,18,-1))
  second = np.load('latent_representations/' + latents[1]).reshape((1,18,-1))
  final_w_vectors = 0.5*first + 0.5*second
  final_w_vectors = np.array(final_w_vectors)
  img = generate_images(averaged_generator_network, final_w_vectors, z = False)[0]
  
  im = Image.fromarray(img)
  im.save("morph.png")


Using TensorFlow backend.


# **Img <-> base64**

In [12]:
import base64

def save_img_from_base64_string(base64_string, path):
    message_bytes = base64.b64decode(base64_string)
    with open(path, "wb") as fh:
        fh.write(message_bytes)

def get_base64_img(path):
  with open(path, "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())
    return encoded_string

# **Run the services**

In [13]:
def run1():
  get_ipython().system_raw('python3 app.py &')

In [14]:
def run2():
  get_ipython().system_raw('./ngrok http 5000 &')

# **Show the ngrok tunnel IP**

In [15]:
def ngrok_tunnel():
  ! curl -s http://localhost:4040/api/tunnels | python3 -c \
      "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

#**REST API**

In [16]:
from flask import Flask, request
import time
import threading

app = Flask(__name__)
is_morph_up_to_date = False


@app.route('/', methods=['GET','POST'])
def index():
  global is_morph_up_to_date
  def do_morph():
    global is_morph_up_to_date
    morphing()
    is_morph_up_to_date = True

  if request.method == 'POST':
    is_morph_up_to_date = False
    save_img_from_base64_string(request.form['your_face'][2:-1], "raw_images/your_face.png")
    save_img_from_base64_string(request.form['star_face'][2:-1], "raw_images/star_face.png")

    thread = threading.Thread(target=do_morph)
    thread.start()
    return 'morphing started'

  elif request.method == 'GET':
    if (is_morph_up_to_date):
      response = {'morph' : str(get_base64_img("morph.png"))}
    else:
      response = {'morph' : 'Not yet'}
    return response

#**Reset and run server**

In [33]:
run1()
run2()

In [34]:
ngrok_tunnel()

https://bafef14e730b.ngrok.io


In [25]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Feb/2021 21:40:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2021 21:40:42] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Feb/2021 21:52:30] "POST / HTTP/1.1" 200 -


Aligning star_face.png ...
Getting landmarks...
Starting face alignment...
Wrote result aligned_images/star_face_01.png
Aligning your_face.png ...
Getting landmarks...
Starting face alignment...
Wrote result aligned_images/your_face_01.png
images_aligned contient 2 images







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
If using Keras pass *_constraint arguments to layers.

58892288/58889256 [==============================] - 0s 0us/step


  0%|          | 0/2 [00:00<?, ?it/s]



Saving mask masks/star_face_01.png
Loading ResNet Model:


...

100%|██████████| 2/2 [03:00<00:00, 90.06s/it]


Loading StyleGAN model from cache/3db02d3ee17602c58c0ab18b2ea88720_karras2019stylegan-ffhq-1024x1024__1_.pkl...


127.0.0.1 - - [11/Feb/2021 21:56:32] "GET / HTTP/1.1" 200 -


StyleGAN loaded & ready for sampling!


127.0.0.1 - - [11/Feb/2021 21:56:52] "GET / HTTP/1.1" 200 -
